# A tutorial on fitting exchange parameters to DFT energies<br/>The Case of Iron

James K. Glasbrenner

November 28, 2017

In [ ]:
# SETUP BLOCK
import pymatgen as pmg
from pymatgen.transformations.standard_transformations import (
    RotationTransformation, SupercellTransformation)
from pymatgen.analysis.structure_analyzer import OrderParameters

## Theoretical background

## Generate the crystal structure using pymatgen

At room temperature, [iron has the following crystal structure][iron-crystal]:

*   Structure: BCC
*   Space group: Im-3m
*   Space group number: 229
*   Lattice constants
    *   $a = 2.8665 \text{ Å}$.
    
This is a straightforward crystal structure and can easily be constructed by hand.
However, we will make use of the Python library `pymatgen`, which allows us to construct and manipulate crystal structures in a systematic and reproducible way.

<!-- Implicit links -->

[iron-crystal]: https://www.webelements.com/iron/crystal_structure.html

In [ ]:
iron = {
    "spacegroup": 229,
    "lattice": {
        "lengths": (2.8665, 2.8665, 2.8665),
        "angles": (90, 90, 90)
    },
    "structure": {
        "species": ["Fe"],
        "basis_coordinates": [[0.00, 0.00, 0.00]]
    },
    "pymatgen": {
        "lattice": None,
        "structure": None
    }
}

In [ ]:
iron["pymatgen"]["lattice"] = pmg.Lattice.from_lengths_and_angles(
    abc=iron["lattice"]["lengths"],
    ang=iron["lattice"]["angles"]
)

iron["pymatgen"]["structure"] = pmg.Structure.from_spacegroup(
    sg=iron["spacegroup"],
    lattice=iron["pymatgen"]["lattice"],
    species=iron["structure"]["species"],
    coords=iron["structure"]["basis_coordinates"]
)

In [ ]:
# Code to find neighbors (needs fixing)

for site_index, site_neighbors in enumerate(all_neighbors):
    site_sublattice_id = pmg_structure[site_index].properties["sublattice"]
    unique_site_distances = unique_sublattice_distances["{0}".format(
        site_sublattice_id)]

    for neighbor in site_neighbors:
        neighbor_sublattice_id = neighbor[0].properties["sublattice"]
        neighbor_distance = neighbor[1]
        unique_neighbor_distances = unique_site_distances["{0}".format(
            neighbor_sublattice_id)]
        new_distance_test = [
            math.isclose(neighbor_distance, x)
            for x in unique_neighbor_distances
        ]
        if not any(new_distance_test):
            unique_neighbor_distances.append(neighbor_distance)

# Sort unique distances between sublattice-sublattice pairs
for sublattice_pairs in unique_sublattice_distances.values():
    for unique_distances in sublattice_pairs.values():
        unique_distances.sort()

all_neighbors_sorted = []
for site_index, site_neighbors in enumerate(all_neighbors):
    site_sublattice_id = pmg_structure[site_index].properties["sublattice"]
    unique_site_distances = unique_sublattice_distances["{0}".format(
        site_sublattice_id)]

    site_neighbors_numbered = []
    for neighbor in site_neighbors:
        neighbor_sublattice_id = neighbor[0].properties["sublattice"]
        neighbor_distance = neighbor[1]
        unique_neighbor_distances = unique_site_distances["{0}".format(
            neighbor_sublattice_id)]
        for neighbor_number, test_distance in enumerate(
                unique_neighbor_distances, start=1):
            if math.isclose(neighbor_distance, test_distance):
                update_row = [x for x in neighbor]
                update_row[1] = tuple([update_row[1], neighbor_number])
        site_neighbors_numbered.append(tuple(update_row))

    site_neighbors_sorted = sorted(
        site_neighbors_numbered,
        key=lambda x: (x[0].properties["sublattice"], x[1][1]))

    all_neighbors_sorted.append(site_neighbors_sorted)

all_neighbors_reduced = []
for site_neighbors in all_neighbors_sorted:
    grouped_neighbors = groupby_list(
        iterable=site_neighbors,
        key=lambda x: (x[0].properties["sublattice"], x[1][1]))
    site_neighbors_reduced = []
    for neighbor in grouped_neighbors:
        neighbor_sublattice_id = neighbor[0][0]
        neighbor_number = neighbor[0][1]
        neighbor_list = neighbor[1]
        neighbor_indices = tuple(x[2] for x in neighbor_list)
        site_neighbors_reduced.append((neighbor_sublattice_id,
                                       neighbor_number, neighbor_indices))
    all_neighbors_reduced.append(site_neighbors_reduced)

build_dataframe = []
for site_index, site_neighbors in enumerate(all_neighbors_reduced):
    site_sublattice_id = pmg_structure[site_index].properties["sublattice"]
    for grouped_neighbors in site_neighbors:
        neighbor_sublattice_id = grouped_neighbors[0]
        neighbor_number = grouped_neighbors[1]
        neighbor_indices = grouped_neighbors[2]
        for neighbor_index in neighbor_indices:
            build_dataframe.append(
                tuple((site_index, neighbor_index, site_sublattice_id,
                       neighbor_sublattice_id, neighbor_number)))
neighbors_df = np.array(build_dataframe, dtype={
    "names": [
        "site_i", "site_j", "sublattice_i", "sublattice_j",
        "neighbor_number"
    ],
    "formats": ["i8", "i8", "i8", "i8", "i8"]
})
neighbors_df = pd.DataFrame(data=neighbors_df)

build_dataframe = []
for (site_sublattice_id,
     neighbor_sublattice_distances) in unique_sublattice_distances.items():
    for (neighbor_sublattice_id,
         neighbor_distances) in neighbor_sublattice_distances.items():
        for neighbor_number, distance in enumerate(neighbor_distances,
                                                   start=1):
            build_dataframe.append(
                tuple((site_sublattice_id, neighbor_sublattice_id,
                       neighbor_number, distance)))
distances_df = np.array(build_dataframe, dtype={
    "names":
    ["sublattice_i", "sublattice_j", "neighbor_number", "distance"],
    "formats": ["i8", "i8", "i8", "f8"]
})
distances_df = pd.DataFrame(data=distances_df)
